In [ ]:
"""
Clean vs Dirty Plate Classification – Final Project

This script trains a binary image classifier to distinguish between clean and dirty plates,
as part of the Kaggle competition "Cleaned vs Dirty" (https://www.kaggle.com/competitions/platesv2).

It performs the following key tasks:
- Loads and augments labeled training data using PyTorch and torchvision
- Trains a ResNet18 model using 5-fold stratified cross-validation
- Stores the best model from each fold and uses their ensemble for prediction
- Applies the ensemble to unlabeled test images and generates a submission CSV
- Trains a second, specialized model for classifying metal bowls (which the original model struggled with)
- Includes an interactive manual review interface to visually inspect and evaluate model predictions

Developed by Abigail Lydick for CS 455 Final Project (UG-9)
"""

'\nClean vs Dirty Plate Classification – Final Project\n\nThis script trains a binary image classifier to distinguish between clean and dirty plates,\nas part of the Kaggle competition "Cleaned vs Dirty" (https://www.kaggle.com/competitions/platesv2).\n\nIt performs the following key tasks:\n- Loads and augments labeled training data using PyTorch and torchvision\n- Trains a ResNet18 model using 5-fold stratified cross-validation\n- Stores the best model from each fold and uses their ensemble for prediction\n- Applies the ensemble to unlabeled test images and generates a submission CSV\n- Trains a second, specialized model for classifying metal bowls (which the original model struggled with)\n- Includes an interactive manual review interface to visually inspect and evaluate model predictions\n\nDeveloped by Abigail Lydick for CS 455 Final Project (UG-9)\n'

In [ ]:
#if using colab, mount your google drive here. Make sure the data set is loaded into your code. Take note of the path to locate it in your drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#import the python packages needed to create the CNN
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset, Dataset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from PIL import Image
import csv

#Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Paths to my google drive
#If using colab, click drive--> folder with dataset --> data --> and then clock path next to each file. It will copy the path to your clipboard. Paste it below inplace of current pathway.
train_dir = '/content/drive/MyDrive/cln/train' #change this path to where you have stored your files
test_dir = '/content/drive/MyDrive/cln/test' #change this path to where you have stored your files

#Transforms
#Defines image preprocessing pipelines using PyTorch's torchvision.transforms for data augmentation and normalization
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

#Test Data
#To load and preprocess test images (which are unlabeled) so they can be passed into your model for inference.
class UnlabeledImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_paths = [
            os.path.join(image_dir, fname)
            for fname in os.listdir(image_dir)
            if fname.lower().endswith(('.png', '.jpg', '.jpeg'))
        ]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, os.path.basename(image_path)

#Load datasets in both the train and test folders
full_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
test_dataset = UnlabeledImageDataset(test_dir, transform=val_transforms)

targets = [label for _, label in full_dataset.samples]
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

models_list = []
accuracies = []

#---------------------------------------------------------------------------------------
#This block is the training core of a k-fold cross-validation loop using PyTorch and a CNN model

#Iterates through the 5 stratified splits created
for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(targets)), targets)):
    print(f"\n--- Fold {fold+1} ---")
#Creates Fold Data Subsets
    train_subset = Subset(full_dataset, train_idx)
    val_subset = Subset(full_dataset, val_idx)

#DataLoaders
    train_loader = DataLoader(train_subset, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=8, shuffle=False)

#Model Set-Up
    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, 1)
    model = model.to(device)

# Loss, Optimizer, Scheduler
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3)

#Track Best Model
    best_acc = 0.0
    best_model_wts = model.state_dict()

#Training Loop
    for epoch in range(10):
        model.train()
        train_loss = 0.0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)

#Determines the average loss of each epic
        avg_loss = train_loss / len(train_loader.dataset)
#--------------------------------------------------------------
        #validate
        model.eval()
        val_preds, val_labels = [], []
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                val_preds.extend(torch.sigmoid(outputs).cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        val_preds_binary = [1 if p > 0.5 else 0 for p in val_preds]
        acc = accuracy_score(val_labels, val_preds_binary)
        print(f"Epoch {epoch+1} | Loss: {avg_loss:.4f} | Val Acc: {acc:.4f}")

        if acc > best_acc:
            best_acc = acc
            best_model_wts = model.state_dict()

        scheduler.step(avg_loss)

    model.load_state_dict(best_model_wts)
    models_list.append(model)
    accuracies.append(best_acc)

print(f"\nAverage Validation Accuracy: {np.mean(accuracies):.4f}")

#inference
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)
all_preds = []
filenames = []

for model in models_list:
    model.eval()
    preds = []
    names = []
    with torch.no_grad():
        for inputs, file_names in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            probs = torch.sigmoid(outputs).cpu().numpy().flatten()
            preds.extend(probs)
            names.extend(file_names)
    all_preds.append(preds)
    filenames = names

#final predictions
final_probs = np.mean(all_preds, axis=0)
final_labels = [1 if p > 0.5 else 0 for p in final_probs]

#create file with predictions
#this is the file that will be submitted at https://www.kaggle.com/competitions/platesv2/submissions
with open("submission.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["id", "label"])
    writer.writerows(zip(filenames, final_labels))

print("\nSubmission file")



--- Fold 1 ---


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1 | Loss: 0.7019 | Val Acc: 0.2500
Epoch 2 | Loss: 0.4285 | Val Acc: 0.3750
Epoch 3 | Loss: 0.2691 | Val Acc: 0.3750
Epoch 4 | Loss: 0.1440 | Val Acc: 0.3750
Epoch 5 | Loss: 0.1174 | Val Acc: 0.3750
Epoch 6 | Loss: 0.0617 | Val Acc: 0.3750
Epoch 7 | Loss: 0.0990 | Val Acc: 0.3750
Epoch 8 | Loss: 0.0690 | Val Acc: 0.3750
Epoch 9 | Loss: 0.0141 | Val Acc: 0.3750
Epoch 10 | Loss: 0.0333 | Val Acc: 0.3750

--- Fold 2 ---


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1 | Loss: 0.6812 | Val Acc: 0.7500
Epoch 2 | Loss: 0.3154 | Val Acc: 0.7500
Epoch 3 | Loss: 0.2118 | Val Acc: 0.8750
Epoch 4 | Loss: 0.0870 | Val Acc: 0.8750
Epoch 5 | Loss: 0.1310 | Val Acc: 0.8750
Epoch 6 | Loss: 0.0447 | Val Acc: 0.7500
Epoch 7 | Loss: 0.0297 | Val Acc: 0.6250
Epoch 8 | Loss: 0.0271 | Val Acc: 0.7500
Epoch 9 | Loss: 0.0122 | Val Acc: 0.7500
Epoch 10 | Loss: 0.0122 | Val Acc: 0.6250

--- Fold 3 ---


...

Epoch 1 | Loss: 0.6922 | Val Acc: 0.6250
Epoch 2 | Loss: 0.3083 | Val Acc: 0.7500
Epoch 3 | Loss: 0.1962 | Val Acc: 0.6250
Epoch 4 | Loss: 0.0845 | Val Acc: 0.6250
Epoch 5 | Loss: 0.0987 | Val Acc: 0.8750
Epoch 6 | Loss: 0.0345 | Val Acc: 0.6250
Epoch 7 | Loss: 0.0486 | Val Acc: 0.7500
Epoch 8 | Loss: 0.0273 | Val Acc: 0.7500
Epoch 9 | Loss: 0.0390 | Val Acc: 0.6250
Epoch 10 | Loss: 0.0969 | Val Acc: 0.8750

--- Fold 4 ---


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1 | Loss: 0.6483 | Val Acc: 0.6250
Epoch 2 | Loss: 0.3949 | Val Acc: 0.3750
Epoch 3 | Loss: 0.1086 | Val Acc: 0.5000
Epoch 4 | Loss: 0.1496 | Val Acc: 0.5000
Epoch 5 | Loss: 0.0744 | Val Acc: 0.5000
Epoch 6 | Loss: 0.1500 | Val Acc: 0.5000
Epoch 7 | Loss: 0.0668 | Val Acc: 0.6250
Epoch 8 | Loss: 0.0491 | Val Acc: 0.6250
Epoch 9 | Loss: 0.0314 | Val Acc: 0.6250
Epoch 10 | Loss: 0.0939 | Val Acc: 0.6250

--- Fold 5 ---


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1 | Loss: 0.6065 | Val Acc: 0.6250
Epoch 2 | Loss: 0.3274 | Val Acc: 0.7500
Epoch 3 | Loss: 0.1755 | Val Acc: 0.7500
Epoch 4 | Loss: 0.2451 | Val Acc: 0.8750
Epoch 5 | Loss: 0.1552 | Val Acc: 0.7500
Epoch 6 | Loss: 0.1518 | Val Acc: 0.8750
Epoch 7 | Loss: 0.0488 | Val Acc: 0.8750
Epoch 8 | Loss: 0.0365 | Val Acc: 0.8750
Epoch 9 | Loss: 0.0149 | Val Acc: 0.7500
Epoch 10 | Loss: 0.0892 | Val Acc: 0.7500

Average Validation Accuracy: 0.7250

Submission file


In [ ]:
# === Train metal-only classifier with oversampling ===
#
# The CNN struggles to understand images of clean metal bowls.
# This portion of code will add the "metal_train" photos into the training algorithm with the 40 original images
#
# ------------------------------------------------------
print("\n Training metal-only classifier on /metal_train with class-balanced sampling...")

metal_train_dir = "/content/drive/MyDrive/metal_train" #be certain this path is correct in your own directory
metal_dataset = datasets.ImageFolder(metal_train_dir, transform=train_transforms)

# Optional: add a validation split
from sklearn.model_selection import train_test_split
from torch.utils.data import WeightedRandomSampler

metal_targets = [label for _, label in metal_dataset.samples]
metal_train_idx, metal_val_idx = train_test_split(
    np.arange(len(metal_dataset)),
    stratify=metal_targets,
    test_size=0.2,
    random_state=42
)

#Oversample clean metal bowls
labels_for_train = [metal_targets[i] for i in metal_train_idx]
class_sample_counts = np.bincount(labels_for_train)
weights = 1. / class_sample_counts
sample_weights = [weights[label] for label in labels_for_train]
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

metal_train_loader = DataLoader(
    Subset(metal_dataset, metal_train_idx),
    batch_size=4,
    sampler=sampler
)
metal_val_loader = DataLoader(
    Subset(metal_dataset, metal_val_idx),
    batch_size=4,
    shuffle=False
)

# Model
metal_model = models.resnet18(pretrained=True)
metal_model.fc = nn.Linear(metal_model.fc.in_features, 1)
metal_model = metal_model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(metal_model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2)

best_acc = 0.0
best_wts = metal_model.state_dict()

for epoch in range(5):
    metal_model.train()
    total_loss = 0.0
    for inputs, labels in metal_train_loader:
        inputs = inputs.to(device)
        labels = labels.float().unsqueeze(1).to(device)
        optimizer.zero_grad()
        outputs = metal_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * inputs.size(0)

    avg_loss = total_loss / len(metal_train_loader.dataset)

    # Validation
    metal_model.eval()
    val_preds, val_labels = [], []
    with torch.no_grad():
        for inputs, labels in metal_val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = metal_model(inputs)
            probs = torch.sigmoid(outputs).cpu().numpy().flatten()
            val_preds.extend(probs)
            val_labels.extend(labels.cpu().numpy())

    val_preds_binary = [1 if p > 0.5 else 0 for p in val_preds]
    acc = accuracy_score(val_labels, val_preds_binary)
    print(f"[Metal Model] Epoch {epoch+1} | Loss: {avg_loss:.4f} | Val Acc: {acc:.4f}")

    if acc > best_acc:
        best_acc = acc
        best_wts = metal_model.state_dict()

    scheduler.step(avg_loss)

metal_model.load_state_dict(best_wts)
print(f"\n Finished training metal-only model with best val acc: {best_acc:.4f}")



🧠 Training metal-only classifier on /metal_train with class-balanced sampling...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[Metal Model] Epoch 1 | Loss: 0.2804 | Val Acc: 1.0000
[Metal Model] Epoch 2 | Loss: 0.2438 | Val Acc: 1.0000
[Metal Model] Epoch 3 | Loss: 0.1676 | Val Acc: 1.0000
[Metal Model] Epoch 4 | Loss: 0.1567 | Val Acc: 1.0000
[Metal Model] Epoch 5 | Loss: 0.0922 | Val Acc: 1.0000

✅ Finished training metal-only model with best val acc: 1.0000


In [ ]:
#import libraies needed to create the manual accuracy checker
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

#Dataset + Prediction Setup
review_dataset = UnlabeledImageDataset(test_dir, transform=val_transforms)
review_loader = DataLoader(review_dataset, batch_size=1, shuffle=False)

review_probs = []
review_preds = []
image_names = []

#Runs predictions for all images in your test/review set--> and then Collects and Prepares the results
with torch.no_grad():
    for inputs, fname in review_loader:
        inputs = inputs.to(device)
        outputs = models_list[-1](inputs)
        probs = torch.sigmoid(outputs).cpu().numpy().flatten()
        preds = [1 if p > 0.5 else 0 for p in probs]
        review_probs.extend(probs)
        review_preds.extend(preds)
        image_names.extend(fname)

#Initializations for the code to begin creating the manual accuracy checker
review_index = 0
correct_count = 0
incorrect_count = 0
manual_review_log = []

#Widgets for the buttons. These will be used to manually determine the accuracy of the CNN
yes_button = widgets.Button(description="✅ Yes")
no_button = widgets.Button(description="❌ No")
progress_label = widgets.Label()
image_box = widgets.Output()
info_box = widgets.Output()
out = widgets.Output()

#defines the progress structure of the manual accuracy checker
def update_progress_label():
    total = len(image_names)
    reviewed = review_index
    left = total - reviewed
    progress_label.value = f"🔄 Reviewed: {reviewed} of {total} | 🔚 {left} left"

#This function pulls the current image to review, correctlty labels the checker, and verifies there are images available to asses
def show_review_image():
    if review_index >= len(image_names):
        with image_box:
            clear_output()
            print(" Manual Review Complete")
            print(f"Correct:   {correct_count}")
            print(f"Incorrect: {incorrect_count}")
            total = correct_count + incorrect_count
            if total > 0:
                print(f"Manual Accuracy: {correct_count / total:.2%}") #done correctly if it finds 744 images and 744 predictions
            else:
                print("No images reviewed.") #if this appears you might need to check the paths to your data. It should find 744 images
        return

#Searches for the images for reviewing and shows them
    update_progress_label()
    with image_box:
        clear_output(wait=True)
        img_path = os.path.join(test_dir, image_names[review_index])
        try:
            pil_img = Image.open(img_path).convert("RGB")
            plt.imshow(pil_img)
            plt.axis("off")
            plt.show()
        except Exception as e:
            print(f"Failed to open image: {img_path}") #if this appears be sure to check pathways in your directory again
            return

#Defining what information will be present in the information box of the accuracy checker.
    with info_box:
        clear_output(wait=True)
        pred = review_preds[review_index]
        prob = review_probs[review_index]
        print(f"Prediction: {'dirty' if pred == 1 else 'clean'} (prob = {prob:.2f})")
        print(f"Image: {image_names[review_index]}")

#Tells the code to move foward through the predictions
def advance():
    global review_index
    review_index += 1
    show_review_image()

#Tells the code what to do after "yes" is clicked
def on_yes_clicked(b):
    global correct_count
    correct_count += 1
    manual_review_log.append(('✓', image_names[review_index], review_preds[review_index], review_probs[review_index]))
    advance()

#Tells the code what to do after "no" is clicked
def on_no_clicked(b):
    global incorrect_count
    incorrect_count += 1
    manual_review_log.append(('✗', image_names[review_index], review_preds[review_index], review_probs[review_index]))
    advance()

#Bind buttons
yes_button.on_click(on_yes_clicked)
no_button.on_click(on_no_clicked)

#Defining the layout of the accuracy checker
display(widgets.VBox([
    widgets.HTML("<h3>🟢 Manual Review of Test Predictions</h3>"),
    progress_label,
    image_box,
    info_box,
    widgets.HBox([yes_button, no_button]),
    out
]))

#Initializes the manual accuracy check screen in a markdown cell
show_review_image()

#--------------------------------------------------------------------------------
#  The markdown cell will show you;
#   1. An image/name of a random plate from the 744 plates in the "train" database
#   2. What the algorithm predicted the plate to be (Clean or Dirty)
#   3. How many images have been reviwed out of 744
#   4. How many images left to review
#   5. A value of how confident the algorithm is in it's prediction
#         0.9 Model is very confident the plate is dirty
#         0.7 Model thinks the plate is probably dirty, but isn't 100% sure
#         0.5 Model is unsure- it is right at the decision threshold
#         0.3 Model thinks the plate is probably clean, but isn't 100% sure
#         0.05 Model is very confident the plate is clean
#
# How to use this feature
#     Yes- click here if the prediction correct and matches the image
#     No- click here if the prediction is incorrect
#
# After assesing all 744 images in the "test" dataset the code will produce;
#   1. The accuracy of the model
#   2. The amount of plates the algorithm correctly identified
#   3. The amount of plates the algorithm incorrectly identified
#
# Example Output:
#     Manual Review Complete
#     Correct:   648
#     Incorrect: 96
#     Manual Accuracy: 87.10%
#---------------------------------------------------------------------------------